# Simple call

In [23]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv(override=True)

messages = [{"role":"user", "content":"Give me 1 sentence differentiating between meiosis and mitosis."}]

# OpenAI
client = OpenAI()
response = client.chat.completions.create(
    messages=messages,
    model='gpt-4.1-nano'
)
result = (f"From gpt-4.1-nano: {response.choices[0].message.content}")
display(Markdown(result))

# Other
client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com/v1")
response = client.chat.completions.create(
    messages=messages,
    model='deepseek-chat'
)
result = f"From deepseek-chat: {response.choices[0].message.content}"
display(Markdown(result))

From gpt-4.1-nano: Meiosis is a cell division process that reduces the chromosome number by half to produce gametes, while mitosis results in two genetically identical diploid daughter cells for growth and repair.

From deepseek-chat: Mitosis produces two genetically identical diploid cells for growth and repair, while meiosis creates four genetically unique haploid cells specifically for sexual reproduction.

# Agent

In [ ]:
from agents import Agent, Runner
from agents import AsyncOpenAI, OpenAIResponsesModel, trace

What is the mechanism by which enzymes lower the activation energy of biochemical reactions?


In [62]:
instruction = "You generate question about science that can be solved by college graduates."
agent = Agent(name="Science Question Agent", instructions=instruction, model='gpt-4.1-nano')
science_question_tool = agent.as_tool(tool_name="question_maker_science", tool_description="Ask a science question.")

In [63]:
instruction = "You generate question about art history that can be solved by college graduates."
agent = Agent(name='Art Question Agent', instructions=instruction, model='gpt-4.1-nano')
art_question_tool = agent.as_tool(tool_name='question_maker_art', tool_description="Ask an art question.")

In [60]:
instruction = "You provide answers to science and art history questions. Your knowledge is a college graduate level."
agent = Agent(name='Answer Provider Agent', instructions=instruction, model='gpt-4.1-nano')
answer_tool = agent.as_tool(tool_name='solution_provider', tool_description="Give an answer.")

In [67]:
lecturer_instructions = """
You give a question and answer. Your goal is to select

Follow these steps carefully:
1. Generate questions: Use question_maker_art and qeustion_maker_science tools to generate 2 different questions.
2. Evaluate: Provide answers to the 2 questions by using solution_provider tool.
3. Report: Publish the questions and answers.

"""
lecturer = Agent(
    name="Lecturer",
    instructions=lecturer_instructions,
    tools = [science_question_tool, art_question_tool, answer_tool],
    model='gpt-4.1-nano'
)

message = "Give me a question and answer so that I can learn something new today."

with trace('QandA2'):
    result = await Runner.run(lecturer, message)

    print(result.final_output)

Here are two questions and their answers to help you learn something new today:

**Art Question:**  
How did the Romantic movement’s emphasis on emotion and individualism manifest in the works of J.M.W. Turner and Eugène Delacroix, and in what ways did their approaches differ in portraying nature and human experience?

**Art Answer:**  
This question invites an analysis of Romanticism's core ideals by comparing Turner’s atmospheric, often tumultuous landscapes and seascapes with Delacroix’s vivid, expressive scenes and figure paintings. Turner’s work emphasizes the sublime power of nature through luminous, dynamic compositions often depicting natural phenomena, while Delacroix channels intense emotional expression and drama through vibrant color and energetic brushwork. Their differing techniques and subject matter illustrate the broad spectrum of Romantic artistic philosophy, with Turner focusing on nature’s awe-inspiring aspect and Delacroix emphasizing passionate human experience.



# Agent (Async)